In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'utilizando dispositivo {device}')

utilizando dispositivo cuda


In [3]:
# transformação para o pre processamento das imagens
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(), # converter pra tensor
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)) #normalização para [-1, 1]
])

In [4]:
dataset = datasets.ImageFolder(root='PetImages', transform=transform)

In [5]:
# definir proporções entre treino e teste
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [6]:
# dividir dados aleatoriamente
train_data, test_data = random_split(dataset, [train_size, test_size])

In [7]:
# criar data loader
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle= False)

In [8]:
print(f'tamanho do conjunto de treino: {len(train_data)} imagens')
print(f'tamanho do conjunto de teste: {len(test_data)} imagens')

tamanho do conjunto de treino: 19998 imagens
tamanho do conjunto de teste: 5000 imagens


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2) # saida de duas classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16) # flatten
        x = self.relu(self.fc1(x)) 
        x = self.fc2(x) #Removemos Softmax porque CrossEntropyLoss já lida com isso
        return x

In [10]:
# criar a rede e mover pra GPU
modelo = CNN().to(device)

# função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo.parameters(), lr = 0.001)

In [11]:
# treino
def train(modelo, train_loader, criterion, optimizer, epochs = 10):
    modelo.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = modelo(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            print(f"Época {epoch + 1} Loss: {running_loss/len(train_loader):.4f}")


In [12]:
# teste
def test(modelo, test_loader):
    modelo.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = modelo(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f"Acurácia: {100 * correct / total:.2f}%")

In [13]:
# treinamento
train(modelo, train_loader, criterion, optimizer, epochs = 20)

Época 1 Loss: 0.0011
Época 1 Loss: 0.0051
Época 1 Loss: 0.0063
Época 1 Loss: 0.0075
Época 1 Loss: 0.0087
Época 1 Loss: 0.0099
Época 1 Loss: 0.0109
Época 1 Loss: 0.0120
Época 1 Loss: 0.0131
Época 1 Loss: 0.0142
Época 1 Loss: 0.0153
Época 1 Loss: 0.0163
Época 1 Loss: 0.0174
Época 1 Loss: 0.0186
Época 1 Loss: 0.0197
Época 1 Loss: 0.0208
Época 1 Loss: 0.0219
Época 1 Loss: 0.0230
Época 1 Loss: 0.0241
Época 1 Loss: 0.0251
Época 1 Loss: 0.0263
Época 1 Loss: 0.0274
Época 1 Loss: 0.0285
Época 1 Loss: 0.0297
Época 1 Loss: 0.0307
Época 1 Loss: 0.0318
Época 1 Loss: 0.0329
Época 1 Loss: 0.0340
Época 1 Loss: 0.0351
Época 1 Loss: 0.0362
Época 1 Loss: 0.0373
Época 1 Loss: 0.0384
Época 1 Loss: 0.0395
Época 1 Loss: 0.0406
Época 1 Loss: 0.0417
Época 1 Loss: 0.0428
Época 1 Loss: 0.0439
Época 1 Loss: 0.0450
Época 1 Loss: 0.0461
Época 1 Loss: 0.0472
Época 1 Loss: 0.0483
Época 1 Loss: 0.0494
Época 1 Loss: 0.0504
Época 1 Loss: 0.0515
Época 1 Loss: 0.0526
Época 1 Loss: 0.0537
Época 1 Loss: 0.0548
Época 1 Loss:

C:\Users\Mateus\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Época 1 Loss: 0.2813
Época 1 Loss: 0.2824
Época 1 Loss: 0.2835
Época 1 Loss: 0.2846
Época 1 Loss: 0.2855
Época 1 Loss: 0.2867
Época 1 Loss: 0.2879
Época 1 Loss: 0.2888
Época 1 Loss: 0.2898
Época 1 Loss: 0.2906
Época 1 Loss: 0.2917
Época 1 Loss: 0.2925
Época 1 Loss: 0.2934
Época 1 Loss: 0.2943
Época 1 Loss: 0.2951
Época 1 Loss: 0.2960
Época 1 Loss: 0.2968
Época 1 Loss: 0.2980
Época 1 Loss: 0.2989
Época 1 Loss: 0.2998
Época 1 Loss: 0.3007
Época 1 Loss: 0.3015
Época 1 Loss: 0.3024
Época 1 Loss: 0.3032
Época 1 Loss: 0.3038
Época 1 Loss: 0.3047
Época 1 Loss: 0.3055
Época 1 Loss: 0.3063
Época 1 Loss: 0.3071
Época 1 Loss: 0.3081
Época 1 Loss: 0.3090
Época 1 Loss: 0.3101
Época 1 Loss: 0.3111
Época 1 Loss: 0.3119
Época 1 Loss: 0.3127
Época 1 Loss: 0.3135
Época 1 Loss: 0.3143
Época 1 Loss: 0.3152
Época 1 Loss: 0.3161
Época 1 Loss: 0.3170
Época 1 Loss: 0.3182
Época 1 Loss: 0.3194
Época 1 Loss: 0.3201
Época 1 Loss: 0.3208
Época 1 Loss: 0.3217
Época 1 Loss: 0.3225
Época 1 Loss: 0.3234
Época 1 Loss:

In [14]:
# teste
test(modelo, test_loader)

Acurácia: 78.76%


In [15]:
torch.save(modelo.state_dict(), "modelo_cachorros_gatos.pth")
print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!
